# FIFA World Cup 2022 Tweets - Limpieza y Preprocesamiento con NLTK

In [24]:
%pip install nltk
%pip install emoji
%pip install pandas


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
import emoji
import sys
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

#  Descargar recursos de NLTK

In [3]:
# --- Descargar recursos de NLTK ---
print("Verificando/Descargando recursos de NLTK ('punkt', 'stopwords')...")
try:
    nltk.data.find('tokenizers/punkt')
except (nltk.downloader.DownloadError, LookupError):
    print("Recurso 'punkt' de NLTK no encontrado. Intentando descargar...")
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except (nltk.downloader.DownloadError, LookupError):
    print("Recurso 'stopwords' de NLTK no encontrado. Intentando descargar...")
    nltk.download('stopwords')

Verificando/Descargando recursos de NLTK ('punkt', 'stopwords')...


# Cargar las stopwords una vez


In [4]:
# Cargar las stopwords una vez
stop_words = set(stopwords.words('english'))
print("Recursos de NLTK 'punkt' y 'stopwords' verificados/descargados.")

Recursos de NLTK 'punkt' y 'stopwords' verificados/descargados.


#  Configuración

In [20]:
# --- Configuración ---
ruta_base = os.path.dirname(os.getcwd())
input_folder = 'data'
input_filename = 'fifa_world_cup_2022_tweets.csv'
input_csv_file = os.path.join(ruta_base,input_folder, input_filename)


output_folder = os.path.join(ruta_base,'data_processed') 
output_filename = 'fifa_tweets_clean.csv'
output_csv_file = os.path.join(output_folder, output_filename)

original_columns_to_keep = [
    'Date Created',
    'Number of Likes',
    'Source of Tweet',
    'Tweet',
    'Sentiment'
]

#  Funciones de procesamiento

In [21]:
def extract_hashtags(tweet):
    """Extrae hashtags de un tweet."""
    if pd.isna(tweet):
        return []
    tweet_str = str(tweet)
    hashtags = re.findall(r'#(\w+)', tweet_str)
    return hashtags

def get_emoji_descriptions(tweet):
    """Convierte emojis en texto descriptivo (ej: ❤️ -> red heart)."""
    if pd.isna(tweet):
        return []
    tweet_str = str(tweet)
    emoji_list_found = emoji.emoji_list(tweet_str)
    descriptions = []
    for emo in emoji_list_found:
        description = emoji.demojize(emo['emoji'], delimiters=("", "")).lower()
        descriptions.append(description)
    return descriptions

def clean_tweet(tweet):
    """Limpia un tweet: elimina URLs, hashtags y emojis, y mantiene menciones sin arroba."""
    if pd.isna(tweet):
        return ""

    text = str(tweet)

    text = re.sub(r'http[s]?://\S+', '', text)
    text = re.sub(r'pic.twitter.com/\S+', '', text)
    text = re.sub(r't.co/\S+', '', text)

    text = re.sub(r'\B@(\w+)', r'\1', text)

    text = re.sub(r'\B#\w+', '', text)

    text = emoji.replace_emoji(text, '')

    text = re.sub(r'[^A-Za-z\s]+', '', text)

    text = re.sub(r'\s+', ' ', text).strip()

    text = text.lower()

    return text

def process_text_for_ml(text):
    """Aplica tokenización, lowercasing y remoción de stopwords."""
    if pd.isna(text) or not isinstance(text, str) or text.strip() == "":
        return []

    text = text.lower()

    tokens = word_tokenize(text)

    #Remover caracteres no alfabéticos y palabras cortas después de tokenizar
    # isalpha() verifica si contiene solo letras, len>1 quita palabras de 1 letra
    tokens = [word for word in tokens if word.isalpha() and len(word) > 1]

    #Remover Stopwords
    tokens = [word for word in tokens if word not in stop_words]

    return tokens


#  Procesamiento principal: Carga y Preprocesamiento


In [22]:
# --- Procesamiento principal: Carga y Preprocesamiento ---
print(f"Cargando dataset desde '{input_csv_file}'...")
try:
    df = pd.read_csv(
        input_csv_file, # Usamos la ruta completa
        encoding='utf-8',
        encoding_errors='replace',
        engine='python',
        on_bad_lines='skip'
    )

    print(f"Dataset cargado exitosamente (usando encoding='utf-8' con manejo de errores, engine='python', saltando líneas con error).")
    print(f"Nota: Caracteres no UTF-8 fueron reemplazados. Algunas líneas con formato incorrecto pueden haber sido omitidas durante la carga.")
    print(f"Número de filas cargadas: {len(df)}")

except FileNotFoundError:
    print(f"Error: El archivo '{input_csv_file}' no fue encontrado.")
    print(f"Asegúrate de que el archivo CSV esté en la carpeta '{input_folder}' dentro del directorio donde ejecutas el script, o proporciona la ruta completa correcta.")
    sys.exit(1)
except Exception as e:
    print(f"Error inesperado al cargar el archivo CSV: {e}")
    sys.exit(1)

# Asegurarse de que las columnas necesarias existen
required_columns = original_columns_to_keep
if not all(col in df.columns for col in required_columns):
    missing = [col for col in required_columns if col not in df.columns]
    print(f"Error: Faltan columnas requeridas en el dataset: {missing}")
    print(f"Columnas disponibles: {df.columns.tolist()}")
    sys.exit(1)


df['Tweet'] = df['Tweet'].fillna('')
df['Sentiment'] = df['Sentiment'].fillna('unknown')

print("Iniciando procesamiento de tweets...")

df['hastag'] = df['Tweet'].astype(str).apply(extract_hashtags)
df['emojis'] = df['Tweet'].astype(str).apply(get_emoji_descriptions)

df['test_clean'] = df['Tweet'].astype(str).apply(clean_tweet)


df['processed_tokens'] = df['test_clean'].astype(str).apply(process_text_for_ml)

print("Procesamiento de tweets completado.")

Cargando dataset desde 'c:\Users\oscar\Documents\Semestre6\discretas\ti2-2025-1-lora_team\data\fifa_world_cup_2022_tweets.csv'...
Dataset cargado exitosamente (usando encoding='utf-8' con manejo de errores, engine='python', saltando líneas con error).
Nota: Caracteres no UTF-8 fueron reemplazados. Algunas líneas con formato incorrecto pueden haber sido omitidas durante la carga.
Número de filas cargadas: 22524
Iniciando procesamiento de tweets...
Procesamiento de tweets completado.


#  Seleccionar y guardar las columnas procesadas ---


In [23]:
# --- Seleccionar y guardar las columnas procesadas ---
print(f"\n--- Guardando dataset procesado ---")

cols_to_save = [col for col in (original_columns_to_keep + ['test_clean', 'hastag', 'emojis', 'processed_tokens']) if col in df.columns]

try:
    os.makedirs(output_folder, exist_ok=True)
    print(f"Carpeta de salida '{output_folder}' asegurada.")
except Exception as e:
    print(f"Error al crear la carpeta de salida '{output_folder}': {e}")

try:
    df[cols_to_save].to_csv(output_csv_file, index=False, encoding='utf-8')
    print(f"Dataset guardado exitosamente en '{output_csv_file}'.")
except Exception as e:
    print(f"Error al guardar el archivo de salida '{output_csv_file}': {e}")
    print("Asegúrate de tener permisos de escritura en la carpeta de salida.")

print("\nScript de carga y preprocesamiento completado.")


--- Guardando dataset procesado ---
Carpeta de salida 'c:\Users\oscar\Documents\Semestre6\discretas\ti2-2025-1-lora_team\data_processed' asegurada.
Dataset guardado exitosamente en 'c:\Users\oscar\Documents\Semestre6\discretas\ti2-2025-1-lora_team\data_processed\fifa_tweets_clean.csv'.

Script de carga y preprocesamiento completado.
